# XGBoost

**Author:** Aniket Patil  
**Notes:** These notebooks are prepared to run locally or in Google Colab.  
If you don't have the dataset, the code will generate a sample dataset automatically.

Run each cell sequentially. If a required library (xgboost/lightgbm) is missing, install it or use the fallback shown.


In [ ]:
# --- Data loading / sample data generation ---
import os
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_PATH = "data.csv"  # Put your dataset here or change path

if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print("Loaded dataset from", DATA_PATH, "shape:", df.shape)
else:
    print("data.csv not found — generating a sample dataset for demo.")
    from sklearn.datasets import make_classification, make_regression
    # For classification notebooks, make_classification will be used; for regression, make_regression will be used.
    # The specific notebook will override the following if needed.
    X, y = make_classification(n_samples=500, n_features=10, n_informative=5, random_state=42)
    import numpy as np
    df = pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])
    df["target"] = y
    print("Generated sample dataset shape:", df.shape)

# Basic split (classification/regression notebooks will adapt as needed)
if "target" not in df.columns:
    raise ValueError("Expected a 'target' column in the dataset. Edit DATA_PATH or generate a dataset with 'target'.")
X = df.drop(columns=["target"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Split: X_train", X_train.shape, "X_test", X_test.shape)


In [ ]:
# --- XGBoost training (tries to import xgboost; falls back to sklearn's GradientBoosting if not installed) ---
try:
    import xgboost as xgb
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Used xgboost package.")
except Exception as e:
    print("xgboost not available or failed:", e)
    from sklearn.ensemble import GradientBoostingClassifier
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Used sklearn.GradientBoostingClassifier as fallback.")

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))
